In [ ]:
# !pip install --upgrade tensorflow
!pip install tensorflow==2.15
!pip install -U tf_keras # Keras 2
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

!pip install transformers datasets tensorflow

In [ ]:
# !pip install --upgrade transformers

In [ ]:
# !pip show tensorflow
# !pip show Keras

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from transformers import TFElectraForSequenceClassification, ElectraTokenizer
from datasets import load_dataset, load_metric
from google.colab import drive
import numpy as np

In [ ]:
drive.mount('/content/drive')

# Saved fine tuned model
load_model_path = '/content/drive/My Drive/W266_Final_Project/Finetune_BASE_GLUE/electra-base-sst-final'

Mounted at /content/drive


In [ ]:
# https://colab.research.google.com/drive/1VoWyzlXZVR5sNygejktJ4Uguw3HIZckb?usp=sharing

# Load ELECTRA BASE fine tuned
tokenizer = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')
model = TFElectraForSequenceClassification.from_pretrained(load_model_path)

# Load and preprocess the GLUE SST dataset
# https://huggingface.co/docs/datasets/v1.1.2/loading_datasets.html
# https://huggingface.co/docs/datasets/en/process
# https://huggingface.co/datasets/nyu-mll/glue
dataset = load_dataset("glue", "sst2")
metric = load_metric("accuracy")

encoded_dataset = dataset.map(lambda examples: tokenizer(examples["sentence"],
                                                         truncation=True,
                                                         padding="max_length",
                                                         return_tensors="tf"),
                              batched=True)

# Convert to TensorFlow dataset
# https://huggingface.co/docs/datasets/v1.16.1/use_dataset.html
validation_data = encoded_dataset["validation"].to_tf_dataset(columns=["attention_mask", "input_ids", "token_type_ids"],
                                                              label_cols=["label"],
                                                              shuffle=False,
                                                              batch_size=32)

# Predict
# https://huggingface.co/docs/datasets/v1.12.0/metrics.html
predictions = model.predict(validation_data)['logits']
predicted_labels = np.argmax(predictions, axis=1)
true_labels = [example['label'] for example in dataset['validation']]
accuracy = metric.compute(predictions=predicted_labels, references=true_labels)

print('Accuracy SST:', accuracy['accuracy'])

- ELECTRA BASE fine tuned on GLUE SST (Stanford Sentiment Treebank) to determine if the sentence is positive or negative in sentence.
- Ran in Google Colab Pro+ on T4 GPU and High RAM
  - System RAM 2.7GB
  - GPU RAM 4.1GB
  - Ran in about 52 seconds
- Accuracy SST: 0.930045871559633
